In [1]:
import sys
sys.path.append("..")
import pickle
arquivo_selic = '/workspaces/Predicoes_macroeconomicas/dados/economic_data_brazil.pkl'
dados_salvos_selic = pickle.load(open(arquivo_selic, 'rb'))

In [2]:
dados_salvos_selic['selic']

2000-01-01    18.94
2000-02-01    18.87
2000-03-01    18.85
2000-04-01    18.62
2000-05-01    18.51
              ...  
2024-02-01    11.15
2024-03-01    11.00
2024-04-01    10.65
2024-05-01    10.46
2024-06-01    10.40
Freq: MS, Name: selic, Length: 294, dtype: float64